In [8]:
import pandas as pd
import chardet
import glob

# 해당폴더 파일명 SOHO로 시작하는 csv파일 경로 리스트로 저장
# dataset/SOHO를 자신의 경로로 바꾸기
data = glob.glob("SOHO_data/SOHO*.csv")
# 경로특수문자 변경
data = [d.replace('\\', '/') for d in data]
df = pd.DataFrame()
for path in data:
    # 인코딩 확인
    encoding = ''
    with open(path, "rb") as f:
        encoding = chardet.detect(f.readline())["encoding"]
    # 아스키코드인 경우 한글로 특정
    if encoding == 'ascii':
        encoding = 'euc-kr'
    # 인코딩이 None인 경우 기본값으로 'utf-8' 사용
    elif encoding is None:
        encoding = 'utf-8'
    
    # 인코딩 적용한 파일 불러오기 및 합치기
    try:
        tmp = pd.read_csv(path, encoding=encoding)
    except UnicodeDecodeError:
        # 다른 인코딩 시도
        try:
            tmp = pd.read_csv(path, encoding='utf-8')
        except UnicodeDecodeError:
            tmp = pd.read_csv(path, encoding='euc-kr')
    df = pd.concat([df, tmp])

print(df.dtypes)

df['YEAR'] = df['STD_YM']//100
df['MONTH'] = df['STD_YM']%100

STD_YM           int64
BLCK_SP_CD       int64
CTPV_CD          int64
CTPV_NM         object
CAT01_CD        object
OPN_CNT          int64
CLSD_CNT         int64
NEW_CNT          int64
MANAGE01_CNT     int64
MANAGE02_CNT     int64
MANAGE03_CNT     int64
MANAGE04_CNT     int64
dtype: object


In [12]:
# 칼럼을 코드북대로 만들어주는 딕셔너리 제작

codeBook = {
    'STD_YM':'time',
    'BLCK_SP_CD':'BlockCode',
    'CTPV_CD':'CityCode',
    'CTPV_NM':'CItyName',
    'CAT01_CD':'IndCode',
    'OPN_CNT':'NumOfOpen',
    'CLSD_CNT':'NumOfClose',
    'NEW_CNT':'NumOfNew',
    'MANAGE01_CNT':'NumUnder6m',
    'MANAGE02_CNT':'NumUnder3y',
    'MANAGE03_CNT':'NumUnder5y',
    'MANAGE04_CNT':'NumOver5y'
}

df.rename(columns=codeBook, inplace=True)

df.head()

,time,BlockCode,CityCode,CItyName,IndCode,NumOfOpen,NumOfClose,NumOfNew,NumUnder6m,NumUnder3y,NumUnder5y,NumOver5y,YEAR,MONTH
0,202211,1,11,서울특별시,F,38088,86,162,994,7260,5357,24477,2022,11
1,202211,1,11,서울특별시,Q,25461,65,78,446,2807,3287,18921,2022,11
2,202211,1,11,서울특별시,O,768,1,2,7,83,100,578,2022,11
3,202211,1,11,서울특별시,L,158382,373,143,1506,21645,33435,101796,2022,11
4,202211,1,11,서울특별시,P,38736,230,375,2017,8808,8498,19413,2022,11


In [13]:
df.to_csv('SOHO_data_tmp.csv')